<a href="https://colab.research.google.com/github/PCHEN3/UvA/blob/main/Quantitave_Fin_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Load packages


In [ ]:
import os
import pandas as pd
import numpy as np
import statsmodels.api as sm
# !pip install plotnine
import plotnine as p9
from matplotlib import pyplot as plt

## Import data

In [ ]:
ff = 'https://raw.githubusercontent.com/PCHEN3/UvA/main/DE_FF_Factors.csv'
ff = pd.read_csv(ff)
d = 'https://raw.githubusercontent.com/PCHEN3/UvA/main/DE_data_long.csv'
d = pd.read_csv(d)
print(ff)
print(d)

## Sorting portfolio

In [ ]:
d['RET11_lag2'] = d.groupby('ISIN')['RET11'].shift(2)
d.head()

def sortPort(x, quantiles = 10):
  ranks = pd.qcut(x, quantiles, labels = False, duplicates = "drop") + 1
  ranks = ranks.apply(lambda x: f'p{int(x)}' if not pd.isnull(x) else x)
  return ranks

d['momport']=d.groupby(['mdate'])['RET11_lag2'].apply(lambda x: sortPort(x))
print(d)

##Calculate momentum portfolio returns

In [ ]:
d = d.groupby(['momport','mdate']).mean()['RET']
d = d.reset_index()
print(d.head())

d = d.pivot(index = 'mdate', columns = 'momport', values = 'RET')
d["WML"] = d["p10"] - d["p1"]
print(d.head())

##Analyze momentum returns

In [ ]:
portfolios = d.merge(ff,on='mdate')
print(portfolios.head(10))

x = portfolios[['MktRf','HML','SMB']]
x = sm.add_constant(x)
reg = sm.OLS(portfolios['WML'],x).fit(cov_type = 'HAC', cov_kwds = {'maxlags':11})
print(reg.summary())

residuals = reg.resid
sigma_epsilon = np.std(residuals)
coefs = reg.params

print('Standard Deviation of residuals = ', sigma_epsilon)
print('Information Ratio = ', coefs[0]/sigma_epsilon*np.sqrt(12))
print('Sharpe Ratio = ', np.sqrt(12)*np.mean(portfolios['WML'])/np.std(portfolios['WML']))


##Plot cumulated returns

In [ ]:
plotdata = portfolios.melt(id_vars = ['mdate'],var_name = "portfolio",  value_name = 'return') 
plotdata = plotdata.merge(ff[['Rf','mdate']],on = 'mdate')
plotdata['mdate'] = plotdata['mdate'].apply(str)

ports = ['WML','MktRf','HML','SMB']
def conditions(plotdata):
    if plotdata['portfolio'] in ports:
      return 1+plotdata['return']+plotdata['Rf']
    if plotdata['portfolio'] not in ports:
      return 1+plotdata['return']

plotdata['cumret'] = plotdata.apply(conditions, axis=1)

initial_inv = plotdata.portfolio.unique()
initial_inv = pd.DataFrame (initial_inv, columns = ['portfolio'])
initial_inv['mdate'], initial_inv['return'], initial_inv['Rf'], initial_inv['cumret'] = ['199102', 'NA', 'NA', 1]

plotdata = pd.concat([initial_inv, plotdata])
plotdata['cumret'] = plotdata.groupby(['portfolio']).cumprod()

plotdata['date'] = plotdata['mdate'].str[0:4].astype(str)+'-'+plotdata['mdate'].str[-2:].astype(str)+'-01'
plotdata['date'] = pd.to_datetime(plotdata['date'])

ports = ['WML','MktRf','HML','SMB','Rf']

plotdata1 = plotdata[plotdata['portfolio'].isin(ports)]
plotdata2 = plotdata[~plotdata['portfolio'].isin(ports)]

from plotnine import *

print(
    ggplot(plotdata1)
    + aes(x = 'date', y = 'cumret', color = 'portfolio')
    + geom_line()
    + xlab('Date')
    + ylab('Portfolio value (USD)')
    + scale_y_log10()
    + scale_x_date(breaks = ('1 year'))
    + theme(axis_text_x = element_text(rotation = 90))
) 

print(
    ggplot(plotdata2)
    + aes(x = 'date', y = 'cumret', color = 'portfolio')
    + geom_line()
    + xlab('Date')
    + ylab('Portfolio value (USD)')
    + scale_y_log10()
    + scale_x_date(breaks = ('1 year'))
    + theme(axis_text_x = element_text(rotation = 90))
) 